# PLOTS FOR PAPER

This notebook creates the plots for the paper. Figures are generated as PDFs in the directory cn.PLOT_DIR.

In [ ]:
import pySubnetSB.constants as cn
from pySubnetSB.network import Network
from pySubnetSB.constraint_benchmark import ConstraintBenchmark
from pySubnetSB.identity_hash_benchmark import IdentityHashBenchmark


import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tellurium as te
from typing import List, Optional, Union

In [ ]:
PLOT_ALL = False  # Do all of the plots

In [ ]:
PLOT_TITLE_FONT_INCREMENT = 2

# Data Preparation

In [ ]:
FULL_STRONG_DF = pd.read_csv(cn.FULL_BIOMODELS_STRONG_PATH).sort_values([cn.FINDER_REFERENCE_NAME, cn.FINDER_TARGET_NAME])
STRONG_DF = pd.read_csv(cn.SUBNET_BIOMODELS_STRONG_PATH).sort_values([cn.FINDER_REFERENCE_NAME, cn.FINDER_TARGET_NAME])
STRONG_DF = STRONG_DF.reset_index()
FULL_WEAK_DF = pd.read_csv(cn.FULL_BIOMODELS_WEAK_PATH).sort_values([cn.FINDER_REFERENCE_NAME, cn.FINDER_TARGET_NAME])
WEAK_DF = pd.read_csv(cn.SUBNET_BIOMODELS_WEAK_PATH).sort_values([cn.FINDER_REFERENCE_NAME, cn.FINDER_TARGET_NAME])
WEAK_DF = WEAK_DF.reset_index()
SUMMARY_DF = pd.read_csv(cn.BIOMODELS_SUMMARY_PATH).sort_values(cn.D_MODEL_NAME)
SUMMARY_DF = SUMMARY_DF.reset_index()

In [ ]:
STRONG_DF.head()

In [ ]:
WEAK_DF.head()

In [ ]:
SUMMARY_DF.head

In [ ]:
def makeMergeColumnName(column:str, is_reference:bool=True)->str:
    """
    Creates column names for merged result of subnet dataframe with summary dataframe.
    """
    if is_reference:
        suffix = "_reference"
    else:
        suffix = "_induced"
    return column + "_reference"

In [ ]:
# Augment the subnet information with the reference num_reaction, num_species
def mergeWithSummary(subnet_df:pd.DataFrame=STRONG_DF)->pd.DataFrame:
    """
    Augment the subnet dataframe with summary information for reference network.
    """
    df = subnet_df.merge(SUMMARY_DF, right_on='model_name', left_on='reference_name',
                        suffixes=["_induced", "_reference"])
    # Clean up the DataFrame
    df = df.reset_index()
    del df['index']
    drops = df['reference_name'] == 'something'
    df = df[~drops]
    # Eliminate duplicate rows
    df['key'] = df['reference_name'] + "_" + df['target_name']  # key for an entry
    keys = list(df['key'])
    duplicate_keys = list(set([k for k in keys if keys.count(k) > 1]))
    for duplicate_key in duplicate_keys:
        duplicate_df = df [ df['key'] == duplicate_key]
        indices = list(duplicate_df.index)
        df = df.drop(indices[1:])
    # Final cleanup
    df = df.reset_index()
    return df
#
df = mergeWithSummary()
assert(len([c for c in df.columns if "_reference" in c]) > 0)
assert(len([c for c in df.columns if "_induced" in c]) > 0)
assert(len(df) <= len(STRONG_DF))
print("OK!")

In [ ]:
STRONG_DF = mergeWithSummary(subnet_df=STRONG_DF)
WEAK_DF = mergeWithSummary(subnet_df=WEAK_DF)

## Check for duplicates

In [ ]:
def checkSubnetDuplicates(subnet_df:pd.DataFrame, is_fix:bool=True)->pd.DataFrame:
    df = subnet_df.copy()
    keys = [r + "_" + t for r, t in zip(df[cn.FINDER_REFERENCE_NAME], df[cn.FINDER_TARGET_NAME])]
    df['key'] = keys
    count_dct = {k: keys.count(k) for k in keys}
    if len(keys) > len(set(keys)):
        for key in[k for k in set(keys) if count_dct[k] > 1]:
            if is_fix:
                sel = df['key'] == key
                idxs = df.index[sel]
                df = df.drop(idxs[:-1])
            print(key + "\n")
            print(df[df['key'] == key])
    else:
        print("No duplicates!")
    return df
        

In [ ]:
print("*** WEAK")
WEAK_DF = checkSubnetDuplicates(WEAK_DF, is_fix=True)
checkSubnetDuplicates(WEAK_DF, is_fix=False)
#
print("\n***STRONG")
STRONG_DF = checkSubnetDuplicates(STRONG_DF, is_fix=True)
checkSubnetDuplicates(STRONG_DF, is_fix=False)

# Helpers

In [ ]:
def makeTestDataFrame():
    df = pd.DataFrame({'reference_name': [0, 1, 1, 2, 2, 2], 'target_name': [0, 1, 1, 2, 2, 2]})
    return df.astype(str)
TEST_NUM_DUPLICATE = 5

In [ ]:
def makeAntimony(model_name:str, is_reference:bool=True, subnet_df:pd.DataFrame=STRONG_DF, is_roadrunner_loadable:bool=False):
    """
    Transforms the string in a "network" cell into an antimony model
    """
    if is_reference:
        name_col = 'reference_name'
        network_col = 'reference_network'
    else:
        name_col = 'target_name'
        network_col = 'induced_network'
    models =  subnet_df[subnet_df[name_col] == model_name][network_col].values
    if len(models) == 0:
        return None
    model = models[0]
    if is_roadrunner_loadable:
        pos = model.index('tions\n')
        model = model[pos+7:]
        model = model.replace('\n', ';1\n')
        model += ";1;"
    return model

# TESTS
model = makeAntimony('BIOMD0000000224', is_roadrunner_loadable=True)
rr = te.loada(model)
model = makeAntimony('BIOMD0000000030')
assert(model is None)
print("OK!")

In [ ]:
def extractBiomodelNum(stg:str)->int:
    """
    Extracts the number from the biomodels name.
    """
    substg = stg[5:]
    pos = np.min([n if c != '0' else 1000 for n, c in enumerate(substg)])
    try:
        result = int(substg[pos:])
    except:
        result = None
    return result

# TESTS
num = extractBiomodelNum('BIOMD0000000030')
assert(num == 30)
num = extractBiomodelNum('BIOMD0000002030')
assert(num == 2030)
print("OK!")

In [ ]:
def checkDuplicates(df:pd.DataFrame, is_print:bool=True)->list:
    """
    Checks if elements are duplicated
    """
    keys = list(df['reference_name'] + df['target_name'])
    duplicates = []
    if len(keys) > len(set(keys)):
        duplicates = [k for k in keys if keys.count(k) > 1]
        if is_print:
            print(f"**Duplicate entries: {duplicates}")
    else:
        if is_print:
            print("**No duplicate entries")
    return duplicates

count = len(checkDuplicates(makeTestDataFrame(), is_print=False))
assert(count == TEST_NUM_DUPLICATE)
print("OK!")

In [ ]:
def removeDuplicates(df:pd.DataFrame)->pd.DataFrame:
    """
    Remove rows where the reference_name + target_name is duplicated.

    Args:
        df: dataframe procesed

    Returns:
        DataFrame w/o duplicates
    """
    keys = np.array(df['reference_name'].astype(str) + df['target_name'].astype(str))
    all_positions = np.array(range(len(keys)))
    drop_idxs = []
    for key in set(keys):
        key_positions = all_positions[keys == key]
        drop_idxs.extend(key_positions[:-1])
    result_df = df.drop(drop_idxs)
    return result_df

# TESTS
df = makeTestDataFrame()
result_df = removeDuplicates(df)
assert(len(result_df) == 3)
print("OK!")   

In [ ]:
def getTargetNames(reference_name:str, subnet_df:pd.DataFrame=STRONG_DF)->List[str]:
    """
    Gets the list of target names for the reference, if any.

    Args:
       reference_name: str

    Returns:
       list-str
    """
    sel = subnet_df["reference_name"] == reference_name
    if np.sum(sel) == 0:
        return None
    target_names = subnet_df[sel]["target_name"].values
    return target_names

# Tests
names = getTargetNames("BIOMD0000000224")
assert(len(names) > 0)
names = getTargetNames("BIOMD000000022x")
assert(names is None)
print("OK!")

In [ ]:
def calculateLog10Probability(prob:Union[float, np.ndarray], min_prob=1e-5)->np.ndarray:
    """
    Calculates -Log10 of probabilities
    """
    if isinstance(prob, float) or isinstance(prob, int):
        new_prob = max(prob, min_prob)
    else:
        new_prob = np.array([max(v, min_prob) for v in prob])
    return -np.log10(new_prob)
        
# Test
result = calculateLog10Probability(0.4)
result = calculateLog10Probability(0)
assert(result == 5)
#
result = calculateLog10Probability([0.5, 0])
assert(result[1] == 5)
print("OK!")

# Scalable Identity Detection

In [ ]:
if PLOT_ALL:
    SCALABLE_IDENTITY_DETECTION_PATH = os.path.join(cn.AUXILIARY_DATA_DIR, "scalable_identity_detection.csv")
    species_nums = range(2, 22, 2)
    reaction_nums = range(2, 22, 2)
    benchmark = IdentityHashBenchmark(species_nums, reaction_nums, num_network=1000)
    if not os.path.isfile(SCALABLE_IDENTITY_DETECTION_PATH):
        df = benchmark.plotHashStatistics(font_size=14, is_plot=False)
        df.to_csv(SCALABLE_IDENTITY_DETECTION_PATH)
    else:
        _ = benchmark.plotHashStatistics(font_size=14, hash_statistics_df=df, is_plot=False)
    path = os.path.join(cn.PLOT_DIR, "scalable_identity_detection.pdf")
    plt.savefig(path)
    plt.show()

# Scalable Subnet Discovery

## Evaluation of constraints

In [ ]:
#if PLOT_ALL:
if True:
    reference_size = 20
    target_size = 100
    fill_size = target_size - reference_size
    benchmark = ConstraintBenchmark(reference_size, fill_size=fill_size, num_iteration=1000)
    _ = benchmark.plotCompareConstraints(is_subnet=True, is_plot=False, font_size=18)
    path = os.path.join(cn.PLOT_DIR, "evaluate_constraints.pdf")
    plt.savefig(path)
    plt.show()

## Effectiveness of constratins

In [ ]:
# Combine figures
if PLOT_ALL:
    from matplotlib.gridspec import GridSpec
    fig = plt.figure(figsize=(20, 10))
    gs = GridSpec(1, 32, figure=fig)
    ax1 = fig.add_subplot(gs[:, 0:10])
    ax2 = fig.add_subplot(gs[:, 10:20])
    ax3 = fig.add_subplot(gs[:, 20:30])
    ax4 = fig.add_subplot(gs[:, 31:32])
    num_iteration = 1000
    font_size = 20
    num_digit = 0
    _ = ConstraintBenchmark.plotHeatmap(range(4, 24, 4), range(10, 110, 10), percentile=50, font_size=font_size,
         ax=ax1, is_no_constraint=True, num_iteration=1, is_cbar=False, title="No constraint", is_plot=False,
        num_digit=num_digit)
    _ = ConstraintBenchmark.plotHeatmap(range(4, 24, 4), range(10, 110, 10), percentile=50, is_contains_reference=True,
        font_size=font_size, num_digit=num_digit,
        ax=ax2, num_iteration=num_iteration, is_cbar=False, is_plot=False, title="constraint w/subnet")
    _ = ConstraintBenchmark.plotHeatmap(range(4, 24, 4), range(10, 110, 10), percentile=50, is_contains_reference=False,
        font_size=font_size, num_digit=num_digit,
        ax=ax3, num_iteration=num_iteration, is_cbar=False, is_plot=False, title="constraint w/o subnet")
    norm = mpl.colors.Normalize(vmin=0, vmax=15)
    cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='Reds'),
                 cax=ax4, orientation='vertical', label='log10 number of assignment pairs')
    #ax4.set_label("log10 number of assignment pairs", size=font_size)
    cbar.set_label(label='log10 number of assignment pairs',size=font_size)
    ax2.set_ylabel("")
    ax2.set_yticklabels([])
    ax3.set_ylabel("")
    ax3.set_yticklabels([])
    path = os.path.join(cn.PLOT_DIR, "scalable_subnet_discovery.pdf")
    plt.savefig(path)
    plt.show()

# Statistical significance of a network

Calculation of the POC of networks with weak and strong identity to the BioModels reference networks (those with number reactions <= 10)

In [ ]:
def plotModelPOC(is_strong:bool=True, is_plot:bool=True, font_size=16):
    if is_strong:
        column = cn.D_PROBABILITY_OF_OCCURRENCE_STRONG
        adjective = 'Strong'
    else:
        column = cn.D_PROBABILITY_OF_OCCURRENCE_WEAK
        adjective = 'Weak'
    pivot_df = SUMMARY_DF.pivot_table(values=column,
                    index=cn.D_NUM_REACTION, columns=cn.D_NUM_SPECIES, aggfunc='median')
    pivot_df = pivot_df.map(lambda x: calculateLog10Probability(x))
    pivot_df.sort_index(level=0, ascending=False, inplace=True)
    if is_plot:
        sns.heatmap(pivot_df, annot=True, fmt="1.0f", cmap="coolwarm", vmin=0, vmax=5,
              annot_kws={'size': font_size},
              cbar_kws={'label': '-log10 Probability of occurrence'})
        #plt.title(f"{adjective} Identity")
        plt.xlabel("number of species", size=font_size)
        plt.ylabel("number of reactions", size=font_size)
# Test
plotModelPOC(is_strong=True, is_plot=False)
print("OK!")

In [ ]:
if PLOT_ALL:
    plotModelPOC(is_strong=True)
    path = os.path.join(cn.PLOT_DIR, "strong_identity_significance_reference_networks.pdf")
    plt.savefig(path)
    # Little difference between strong and weak identity
    #plt.figure()
    #plotModelPOC(is_strong=False)

# Subnet Discovery in BioModels

In [ ]:
def heatmapCount(induced_df, title:str="", is_plot:bool=True, is_count:bool=True, ax=None,
        network_column=cn.FINDER_REFERENCE_NAME, vmax:int=-1, fmt="", font_size=8):
    """
    Counts occurrences of reference networks in induced_df or the number of induced networks.
    Args:
        is_count (bool): count distinct values of network_count; if False, calculate ratio of all occurrences to distinct
    """
    # Calculate count of networks
    count_df = induced_df.copy()
    count_df = count_df[[network_column, cn.D_NUM_SPECIES, cn.D_NUM_REACTION]]
    count_df = count_df.drop_duplicates()
    #
    if is_count:
        # Count sizes of reference networks
        plot_df = count_df
        bar_label = 'count'
        if len(fmt) == 0:
            fmt="1.0f"
        if vmax < 0:
            vmax = 10
    else:
        # Count occurrence of induced networks
        plot_df = induced_df
        bar_label = 'ratio'
        if len(fmt) == 0:
            fmt="1.1f"
        if vmax < 0:
            vmax = 100
    network_ser = count_df.groupby([cn.D_NUM_SPECIES, cn.D_NUM_REACTION]).count()[network_column]
    induced_ser = induced_df.groupby([cn.D_NUM_SPECIES, cn.D_NUM_REACTION]).count()[network_column]
    if is_count:
        plot_ser = network_ser
    else:
        plot_ser = induced_ser/network_ser
    num_species = [x[0] for x in network_ser.index]
    num_reactions = [x[1] for x in network_ser.index]
    df = pd.DataFrame({cn.D_NUM_SPECIES: num_species, cn.D_NUM_REACTION: num_reactions, 'count': plot_ser.values})
    pivot_df = df.pivot_table(values='count', index=cn.D_NUM_REACTION, columns=cn.D_NUM_SPECIES)
    pivot_df.sort_index(level=0, ascending=False, inplace=True)
    if is_plot:
        if ax is None:
            _, ax = plt.subplots(1)
        _ = sns.heatmap(pivot_df, annot=True, fmt=fmt, cmap="coolwarm", vmin=0, vmax=vmax,
              annot_kws={'size': font_size}, ax=ax,
              cbar_kws={'label': bar_label})
        ax.figure.axes[-1].yaxis.label.set_size(font_size)
        cbar_ticklabels = ax.figure.axes[-1].get_yticklabels()
        ax.figure.axes[-1].set_yticklabels(cbar_ticklabels, size=font_size)
        ax.set_title(title, size=font_size+PLOT_TITLE_FONT_INCREMENT)
        ax.set_xlabel("number of species", size=font_size)
        ax.set_ylabel("number of reactions", size=font_size)
        xticklabels = ax.get_xticklabels()
        ax.set_xticklabels(xticklabels, size=font_size)
        yticklabels = ax.get_yticklabels()
        ax.set_yticklabels(yticklabels, size=font_size)
# Test
heatmapCount(STRONG_DF, title="", is_plot=False, is_count=True, font_size=14)
print("OK!")

## Reference networks that induce networks

In [ ]:
if False:
    title = "Sizes of Inducing Reference Networks"
    title = ""
    heatmapCount(STRONG_DF, title=title, is_plot=True, network_column=cn.FINDER_REFERENCE_NETWORK,
                 is_count=True, font_size=14)
    path = os.path.join(cn.PLOT_DIR, "all_reference_network_count.pdf")
    plt.savefig(path)

In [ ]:
constrained_strong_df = STRONG_DF.copy()
# Remove small reference networks
sel = [s > 3 and r > 3 for s, r in zip(constrained_strong_df[cn.D_NUM_SPECIES], constrained_strong_df[cn.D_NUM_REACTION])]
CONSTRAINED_STRONG_DF = constrained_strong_df[sel]

In [ ]:
if False:
    title = "Reference Size"
    title = ""
    heatmapCount(CONSTRAINED_STRONG_DF, title=title, is_plot=True, network_column=cn.FINDER_REFERENCE_NETWORK,
                 is_count=True, font_size=14)
    path = os.path.join(cn.PLOT_DIR, "filtered_reference_network_count.pdf")
    plt.savefig(path)

In [ ]:
if False:
    title = "Induced Networks per Reference Network By Reference Size"
    title = ""
    heatmapCount(CONSTRAINED_STRONG_DF, title=title,
                 network_column=cn.FINDER_REFERENCE_NETWORK,
                 is_count=False, vmax=10, font_size=14,
                 is_plot=True)
    path = os.path.join(cn.PLOT_DIR, "filtered_induced_network_reference.pdf")
    plt.savefig(path)

In [ ]:
if PLOT_ALL:
    _, axes = plt.subplots(1, 2, figsize=(20, 10))
    font_size = 20
    title = "count of reference networks"
    heatmapCount(STRONG_DF, title=title, is_plot=True, network_column=cn.FINDER_REFERENCE_NETWORK,
                 is_count=True, font_size=font_size, ax=axes[0])
    title = "induced network per reference network"
    heatmapCount(STRONG_DF, title=title,
                 network_column=cn.FINDER_REFERENCE_NETWORK,
                 is_count=False, vmax=10, font_size=font_size, ax=axes[1],
                 is_plot=True)
    path = os.path.join(cn.PLOT_DIR, "biomodels_results.pdf")
    plt.savefig(path)

In [ ]:
print(f"Number of distinct strong reference models: {len(set(STRONG_DF[cn.FINDER_REFERENCE_NAME]))}")
print(f"Number of distinct weak reference models: {len(set(WEAK_DF[cn.FINDER_REFERENCE_NAME]))}")
print(f"Number of distinct strong reference models processed: {len(set(FULL_STRONG_DF[cn.FINDER_REFERENCE_NAME]))}")
print(f"Number of distinct weak reference models processed: {len(set(FULL_WEAK_DF[cn.FINDER_REFERENCE_NAME]))}")
print(f"Number of distinct strong target models: {len(set(STRONG_DF[cn.FINDER_TARGET_NAME]))}")
print(f"Number of distinct weak target models processed: {len(set(FULL_WEAK_DF[cn.FINDER_TARGET_NAME]))}")
print(f"Number of distinct strong target models processed: {len(set(FULL_STRONG_DF[cn.FINDER_TARGET_NAME]))}")
print(f"Number of distinct weak target models: {len(set(WEAK_DF[cn.FINDER_TARGET_NAME]))}")
print(f"Number of strong pairs evaluated: {len(FULL_STRONG_DF)}")
print(f"Number of weak pairs evaluated: {len(FULL_WEAK_DF)}")
print(f"Number of strong induced subnsets: {len(STRONG_DF)}")
print(f"Number of weak induced subnsets: {len(WEAK_DF)}")

In [ ]:
if False:
    heatmapPOC(STRONG_DF, 'probability_of_occurrence_strong_induced', title="Strong, Induced, POC", is_plot=True)
    plt.figure()
    heatmapPOC(WEAK_DF, 'probability_of_occurrence_strong_induced', title="Weak, Induced, POC", is_plot=True)

## Target networks with induced networks

In [ ]:
strong_df = CONSTRAINED_STRONG_DF[[cn.FINDER_REFERENCE_NAME, cn.FINDER_TARGET_NAME]]
strong_df = strong_df.merge(SUMMARY_DF, right_on='model_name', left_on='target_name')
strong_df = strong_df[[cn.FINDER_REFERENCE_NAME, cn.FINDER_TARGET_NAME, cn.D_NUM_REACTION, cn.D_NUM_SPECIES]]

In [ ]:
if False:
    title = "Target Sizes"
    title = ""
    heatmapCount(strong_df, title=title, is_plot=True, network_column=cn.FINDER_TARGET_NAME,
                 is_count=True, vmax=5)
    path = os.path.join(cn.PLOT_DIR, "target_network_count.pdf")
    plt.savefig(path)

In [ ]:
if False:
    title = "Induced Networks per Target Network By Target Size"
    title = ""
    heatmapCount(strong_df, title=title, network_column=cn.FINDER_TARGET_NAME,
                 is_count=False, vmax=5, fmt="1.0f",
                 is_plot=True)
    path = os.path.join(cn.PLOT_DIR, "induced_network_target.pdf")
    plt.savefig(path)

In [ ]:
## Weak vs. Strong

In [ ]:
len(STRONG_DF), len(WEAK_DF)

# Reference network details

In [ ]:
# Print all of the reference models that appear in targets
names = list(set(CONSTRAINED_STRONG_DF[cn.FINDER_REFERENCE_NAME]))
names = list(set(STRONG_DF[cn.FINDER_REFERENCE_NAME]))
names.sort()
print(f"Number of distinct networks: {len(names)}\n")
for name in names:
    result = makeAntimony(name)
    if result is not None:
        print(result + '\n\n')